# Készíts egy osztályozási modellt: Finom ázsiai és indiai konyhák


## Konyhai osztályozók 2

Ebben a második osztályozási leckében `további módszereket` fogunk megvizsgálni a kategóriák szerinti adatok osztályozására. Megismerjük azt is, hogy milyen következményekkel járhat az egyik osztályozó választása a másikkal szemben.

### [**Előzetes kvíz**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Előfeltétel**

Feltételezzük, hogy elvégezted az előző leckéket, mivel néhány korábban tanult fogalmat tovább fogunk vinni.

Ehhez a leckéhez a következő csomagokra lesz szükségünk:

-   `tidyverse`: A [tidyverse](https://www.tidyverse.org/) egy [R csomaggyűjtemény](https://www.tidyverse.org/packages), amely a data science-t gyorsabbá, egyszerűbbé és szórakoztatóbbá teszi!

-   `tidymodels`: A [tidymodels](https://www.tidymodels.org/) keretrendszer egy [csomaggyűjtemény](https://www.tidymodels.org/packages/) modellezéshez és gépi tanuláshoz.

-   `themis`: A [themis csomag](https://themis.tidymodels.org/) további recepteket biztosít a kiegyensúlyozatlan adatok kezeléséhez.

A következő módon telepítheted őket:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Alternatívaként az alábbi szkript ellenőrzi, hogy rendelkezel-e a modul elvégzéséhez szükséges csomagokkal, és ha hiányoznak, telepíti őket.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Egy osztályozási térkép**

Az [előző leckében](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1) megpróbáltuk megválaszolni a kérdést: hogyan válasszunk több modell közül? Nagy mértékben függ az adatok jellemzőitől és attól, hogy milyen típusú problémát szeretnénk megoldani (például osztályozás vagy regresszió).

Korábban megtanultuk, hogy milyen lehetőségek állnak rendelkezésre az adatok osztályozására a Microsoft csalólapja segítségével. A Python gépi tanulási keretrendszere, a Scikit-learn, egy hasonló, de részletesebb csalólapot kínál, amely tovább segíthet az osztályozók (más néven becslők) szűkítésében:

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Tipp: [nézd meg ezt a térképet online](https://scikit-learn.org/stable/tutorial/machine_learning_map/), és kattints az útvonalak mentén, hogy elolvasd a dokumentációt.
>
> A [Tidymodels referenciaoldal](https://www.tidymodels.org/find/parsnip/#models) szintén kiváló dokumentációt nyújt a különböző modellfajtákról.

### **A terv** 🗺️

Ez a térkép nagyon hasznos, ha már tisztán látod az adataidat, mivel az útvonalait követve eljuthatsz egy döntéshez:

-   Több mint 50 mintánk van

-   Egy kategóriát szeretnénk előrejelezni

-   Címkézett adataink vannak

-   Kevesebb mint 100 ezer mintánk van

-   ✨ Választhatunk egy Linear SVC-t

-   Ha ez nem működik, mivel numerikus adataink vannak

    -   Kipróbálhatunk egy ✨ KNeighbors Classifiert

        -   Ha ez sem működik, próbáljuk ki a ✨ SVC-t és a ✨ Ensemble Classifiert

Ez egy nagyon hasznos útvonal, amit érdemes követni. Most pedig vágjunk bele a [tidymodels](https://www.tidymodels.org/) modellezési keretrendszer használatába: egy következetes és rugalmas R csomaggyűjtemény, amelyet a jó statisztikai gyakorlat előmozdítására fejlesztettek ki 😊.

## 2. Az adatok felosztása és az egyensúlyhiányos adathalmaz kezelése

Az előző leckékből megtanultuk, hogy a konyháink között volt néhány közös összetevő. Emellett az egyes konyhák száma között elég egyenlőtlen volt az eloszlás.

Ezeket a következő módon kezeljük:

-   Eldobjuk azokat a leggyakoribb összetevőket, amelyek zavart okoznak az eltérő konyhák között, a `dplyr::select()` használatával.

-   Egy `recipe`-t használunk, amely előfeldolgozza az adatokat, hogy modellezésre készen álljanak, egy `over-sampling` algoritmus alkalmazásával.

Az előző leckében már foglalkoztunk ezekkel, így ez most gyerekjáték lesz 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Az egyensúlytalan adatok kezelése

Az egyensúlytalan adatok gyakran negatívan befolyásolják a modell teljesítményét. Sok modell akkor működik a legjobban, ha a megfigyelések száma egyenlő, ezért hajlamosak nehezen boldogulni az egyensúlytalan adatokkal.

Két fő módja van az egyensúlytalan adathalmazok kezelésének:

-   megfigyelések hozzáadása a kisebbségi osztályhoz: `Túlmintavételezés` (Over-sampling), például a SMOTE algoritmus használatával, amely szintetikusan generál új példákat a kisebbségi osztályból ezeknek az eseteknek a legközelebbi szomszédai alapján.

-   megfigyelések eltávolítása a többségi osztályból: `Alulminta-vételezés` (Under-sampling)

Az előző leckénkben bemutattuk, hogyan lehet egyensúlytalan adathalmazokat kezelni egy `recept` segítségével. A recept egy olyan tervrajzként fogható fel, amely leírja, hogy milyen lépéseket kell alkalmazni egy adathalmazon ahhoz, hogy az elemzésre készen álljon. A mi esetünkben azt szeretnénk elérni, hogy a `tréning halmazunkban` az egyes konyhák száma egyenletesen legyen elosztva. Vágjunk is bele!


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Most már készen állunk a modellek tanítására 👩‍💻👨‍💻!

## 3. Több mint multinomiális regressziós modellek

Az előző leckében multinomiális regressziós modelleket vizsgáltunk. Nézzünk meg néhány rugalmasabb modellt az osztályozáshoz.

### Támogató vektorgépek

Az osztályozás kontextusában a `Támogató vektorgépek` egy gépi tanulási technika, amely arra törekszik, hogy megtalálja azt a *hipersíkot*, amely "legjobban" elválasztja az osztályokat. Nézzünk egy egyszerű példát:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


#### Lineáris Támogató Vektor Osztályozó

A Támogató Vektor Clustering (SVC) a Támogató Vektor gépek (Support-Vector Machines) gépi tanulási technikák családjának egyik ága. Az SVC-ben a hiper-síkot úgy választják meg, hogy az `legtöbb` tanuló mintát helyesen elkülönítse, de `néhány` megfigyelést hibásan osztályozhat. Azáltal, hogy néhány pontot megengedünk a rossz oldalon, az SVM robusztusabbá válik a kiugró értékekkel szemben, így jobban általánosít az új adatokra. Az ezt a megsértést szabályozó paramétert `költségnek` (cost) nevezik, amelynek alapértelmezett értéke 1 (lásd: `help("svm_poly")`).

Hozzunk létre egy lineáris SVC-t azzal, hogy a polinomiális SVM modellben `degree = 1` értéket állítunk be.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Most, hogy a *workflow*-ba foglaltuk az előfeldolgozási lépéseket és a modell specifikációját, folytathatjuk a lineáris SVC betanításával, és közben kiértékelhetjük az eredményeket. A teljesítménymutatókhoz hozzunk létre egy metrikakészletet, amely értékelni fogja: `accuracy`, `sensitivity`, `Positive Predicted Value` és `F Measure`.

> Az `augment()` oszlop(ok)at fog hozzáadni az előrejelzésekhez a megadott adatokhoz.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Támogató vektorgép

A támogató vektorgép (SVM) a támogató vektor osztályozó kiterjesztése, amely lehetővé teszi a nemlineáris határvonalak kezelését az osztályok között. Lényegében az SVM-ek a *kernel trükköt* használják, hogy kibővítsék a jellemzőteret, és alkalmazkodjanak az osztályok közötti nemlineáris kapcsolatokhoz. Az SVM-ek által használt egyik népszerű és rendkívül rugalmas kernel függvény a *Radiális bázisfüggvény.* Nézzük meg, hogyan teljesít az adatainkon.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Sokkal jobb 🤩!

> ✅ Kérlek, nézd meg:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> további olvasmányként.

### Legközelebbi szomszéd osztályozók

A *K*-legközelebbi szomszéd (KNN) egy olyan algoritmus, amelyben minden megfigyelést a többi megfigyeléshez való *hasonlósága* alapján jósolunk meg.

Illesszünk egyet az adatainkra.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Úgy tűnik, hogy ez a modell nem teljesít túl jól. Valószínűleg a modell paramétereinek módosítása (lásd `help("nearest_neighbor")`) javíthatja a teljesítményt. Mindenképpen érdemes kipróbálni.

> ✅ Kérlek, nézd meg:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> hogy többet megtudj a *K*-legközelebbi szomszéd (K-Nearest Neighbors) osztályozókról.

### Együttes osztályozók

Az együttes algoritmusok úgy működnek, hogy több alapbecslőt kombinálnak egy optimális modell létrehozásához, akár:

`bagging`: egy *átlagoló függvény* alkalmazásával az alapmodellek gyűjteményére

`boosting`: olyan modellek sorozatának építésével, amelyek egymásra épülnek a prediktív teljesítmény javítása érdekében.

Kezdjük egy Random Forest modellel, amely egy nagy döntési fa gyűjteményt épít, majd egy átlagoló függvényt alkalmaz, hogy egy jobb általános modellt hozzon létre.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Szép munka 👏!

Próbáljunk ki egy Boosted Tree modellt is.

A Boosted Tree egy olyan együttes módszert határoz meg, amely egy sor egymást követő döntési fát hoz létre, ahol minden fa az előző fák eredményeire épül, hogy fokozatosan csökkentse a hibát. Azokra az elemekre összpontosít, amelyeket helytelenül osztályoztak, és a következő osztályozó illesztését úgy állítja be, hogy kijavítsa azokat.

Különböző módokon lehet illeszteni ezt a modellt (lásd `help("boost_tree")`). Ebben a példában Boosted fákat fogunk illeszteni az `xgboost` motor segítségével.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Kérlek nézd meg:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Az AdaBoost modellt tárgyalja, amely jó alternatívája az xgboost-nak.
>
> hogy többet megtudj az Ensemble osztályozókról.

## 4. Extra - több modell összehasonlítása

Ebben a laborban elég sok modellt illesztettünk 🙌. Fárasztó vagy időigényes lehet sok munkafolyamatot létrehozni különböző előfeldolgozók és/vagy modell specifikációk alapján, majd egyenként kiszámítani a teljesítménymutatókat.

Nézzük meg, hogy tudjuk-e ezt egyszerűsíteni egy olyan függvény létrehozásával, amely egy listányi munkafolyamatot illeszt az edzési adathalmazra, majd visszaadja a teljesítménymutatókat a teszt adathalmaz alapján. Ehhez használni fogjuk a [purrr](https://purrr.tidyverse.org/) csomagból a `map()` és `map_dfr()` függvényeket, hogy függvényeket alkalmazzunk a lista minden elemére.

> A [`map()`](https://purrr.tidyverse.org/reference/map.html) függvények lehetővé teszik, hogy sok for ciklust helyettesíts olyan kóddal, amely tömörebb és könnyebben olvasható. A legjobb hely, ahol a [`map()`](https://purrr.tidyverse.org/reference/map.html) függvényekről tanulhatsz, az az [iteráció fejezet](http://r4ds.had.co.nz/iteration.html) az R for Data Science könyvben.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


A [**workflowset**](https://workflowsets.tidymodels.org/) csomag lehetővé teszi a felhasználók számára, hogy sok modellt hozzanak létre és könnyen illesszenek, de elsősorban olyan mintavételi technikákhoz lett tervezve, mint például a `keresztvalidáció`, amelyet még nem tárgyaltunk.

## **🚀Kihívás**

Ezeknek a technikáknak mindegyike számos paraméterrel rendelkezik, amelyeket módosíthatsz, például az SVM-eknél a `cost`, a KNN-nél a `neighbors`, vagy a Random Forestnél az `mtry` (véletlenszerűen kiválasztott prediktorok).

Kutatás során nézd meg az egyes modellek alapértelmezett paramétereit, és gondold át, hogy ezek módosítása mit jelentene a modell minősége szempontjából.

Ha többet szeretnél megtudni egy adott modellről és annak paramétereiről, használd a következőt: `help("model")`, például `help("rand_forest")`.

> A gyakorlatban általában *becsüljük* a *legjobb értékeket* úgy, hogy sok modellt edzünk egy `szimulált adathalmazon`, és mérjük, hogy ezek a modellek mennyire teljesítenek jól. Ezt a folyamatot **hangolásnak** nevezzük.

### [**Előadás utáni kvíz**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Áttekintés és önálló tanulás**

Ezekben a leckékben sok szakzsargon található, ezért szánj egy percet arra, hogy átnézd [ezt a listát](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) a hasznos terminológiáról!

#### KÖSZÖNET ILLETI:

[`Allison Horst`](https://twitter.com/allison_horst/) az elképesztő illusztrációkért, amelyek barátságosabbá és vonzóbbá teszik az R-t. További illusztrációkat találhatsz a [galériájában](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) és [Jen Looper](https://www.twitter.com/jenlooper) az eredeti Python verzió elkészítéséért ♥️

Boldog tanulást kívánok,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Illusztráció: @allison_horst</figcaption>



---

**Felelősségkizárás**:  
Ez a dokumentum az [Co-op Translator](https://github.com/Azure/co-op-translator) AI fordítási szolgáltatás segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális, emberi fordítást igénybe venni. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy téves értelmezésekért.
